In [3]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np

In [182]:
df_spectra=pd.read_csv("spectrum_energy_intensity.csv",index_col=[0,1])
df_input=pd.read_csv("input_data_table.csv",index_col=0)
df_input.head()


,k6a1,k6a2,k11,k12,k9a1,k9a2
0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
1,-0.5,-0.5,-0.5,-0.5,-0.5,0.0
2,-0.5,-0.5,-0.5,-0.5,-0.5,0.5
3,-0.5,-0.5,-0.5,-0.5,0.0,-0.5
4,-0.5,-0.5,-0.5,-0.5,0.0,0.0


In [209]:
test3=df_spectra.groupby(level=1).size()[0] #this gives the number of spectra in the dataset
print(test3)

#this works to get a matrix for each spectrum 
test2=df_spectra[df_spectra.index.isin(["spectrum0"],level=0)]["Energy"].to_numpy()
print(test2)

test4=df_input[df_input.index.isin([0],level=0)].to_numpy()[0]
print(test4)

470
[-0.758 -0.69  -0.612 -0.472 -0.406 -0.328 -0.152 -0.068 -0.014  0.134
  0.222  0.294  0.472  0.524  0.602  0.748  0.824  0.882]
[-0.5 -0.5 -0.5 -0.5 -0.5 -0.5]


In [210]:
list_spectra_Energy_array=[]
list_spectra_Intensity_array=[]
list_input=[]
#list_k6a1=df_input["k6a1"].to_numpy()
#list_k6a2=df_input["k6a2"].to_numpy()
#list_k11=df_input["k11"].to_numpy()
#list_k12=df_input["k12"].to_numpy()
#list_k9a1=df_input["k9a1"].to_numpy()
#list_k9a2=df_input["k9a2"].to_numpy()
for i in range(df_spectra.groupby(level=1).size()[0]):
    list_spectra_Energy_array.append(df_spectra[df_spectra.index.isin(["spectrum"
                                                                       +str(i)],level=0)]["Energy"].to_numpy())
    list_spectra_Intensity_array.append(df_spectra[df_spectra.index.isin(["spectrum"
                                                                          +str(i)],level=0)]["Intensity"].to_numpy())
    list_input.append(df_input[df_input.index.isin([i],level=0)].to_numpy()[0])
    

In [211]:
df_training=pd.DataFrame({'input':list_input,'Intensity':list_spectra_Intensity_array,'Energy':list_spectra_Energy_array})

In [212]:
df_training.head()

,input,Intensity,Energy
0,"[-0.5, -0.5, -0.5, -0.5, -0.5, -0.5]","[5.20690186, 6.88780025, 3.5507148, 6.73106847...","[-0.758, -0.69, -0.612, -0.47200000000000003, ..."
1,"[-0.5, -0.5, -0.5, -0.5, -0.5, 0.0]","[3.74488966, 23.8435403, 2.65115834, 11.044500...","[-0.6920000000000001, -0.574, -0.278, -0.048, ..."
2,"[-0.5, -0.5, -0.5, -0.5, -0.5, 0.5]","[1.93005472, 1.26134031, 5.11297673, 7.1436760...","[-0.9279999999999999, -0.8740000000000001, -0...."
3,"[-0.5, -0.5, -0.5, -0.5, 0.0, -0.5]","[1.91987695, 1.29420517, 5.05196242, 7.1955590...","[-0.9259999999999999, -0.878, -0.778, -0.71200..."
4,"[-0.5, -0.5, -0.5, -0.5, 0.0, 0.0]","[3.03383711, 23.8370405, 3.08890517, 10.074627...","[-0.672, -0.56, -0.262, -0.03, 0.046, 0.614, 0..."


In [47]:
def create_model(my_learning_rate, my_feature_layer,unit_layer_1=10,unit_layer_2=12,regulations=0.04):
    """Create and compile a regression model."""
    model = tf.keras.models.Sequential()
    # Add the layer containing the feature columns to the model.
    model.add(my_feature_layer)

    # Describe the topography of the model by calling the tf.keras.layers.Dense
    # method once for each layer. We've specified the following arguments:
    #   * units specifies the number of nodes in this layer.
    #   * activation specifies the activation function (Rectified Linear Unit).
    #   * name is just a string that can be useful when debugging.

    # Define the first hidden layer with 10 nodes.   
    model.add(tf.keras.layers.Dense(units=unit_layer_1, 
                                  activation='relu', 
                                  kernel_regularizer=tf.keras.regularizers.l2(regulations),
                                  name='Hidden1'))
  
    # Define the second hidden layer with 12 nodes. 
    model.add(tf.keras.layers.Dense(units=unit_layer_2, 
                                  activation='relu', 
                                  kernel_regularizer=tf.keras.regularizers.l2(regulations),
                                  name='Hidden2'))

    # Define the output layer.
    model.add(tf.keras.layers.Dense(units=1,  
                                    name='Output'))                              
  
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.MeanSquaredError()])

    return model



In [48]:
def train_model(model, dataset, epochs, label_name,
                batch_size=None):
    """Train the model by feeding it data."""

    # Split the dataset into features and label.
    features = {name:np.array(value) for name, value in dataset.items()}
    label = np.array(features.pop(label_name))
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=True) 

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch
  
    # To track the progression of training, gather a snapshot
    # of the model's mean squared error at each epoch. 
    hist = pd.DataFrame(history.history)
    mse = hist["mean_squared_error"]

    return epochs, mse